source : https://nominatim.org/release-docs/latest/api/Search/

https://medium.com/@adri.espejo/getting-started-with-openstreetmap-nominatim-api-e0da5a95fc8a

In [1]:
import pandas as pd
import numpy as np
import requests

hdb_string = './resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv'

C:\Users\Randy\anaconda3\envs\IBM Data Science Project\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Randy\anaconda3\envs\IBM Data Science Project\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\Randy\anaconda3\envs\IBM Data Science Project\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
df = pd.read_csv(hdb_string)

In [3]:
df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113959 entries, 0 to 113958
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   month                113959 non-null  object 
 1   town                 113959 non-null  object 
 2   flat_type            113959 non-null  object 
 3   block                113959 non-null  object 
 4   street_name          113959 non-null  object 
 5   storey_range         113959 non-null  object 
 6   floor_area_sqm       113959 non-null  float64
 7   flat_model           113959 non-null  object 
 8   lease_commence_date  113959 non-null  int64  
 9   remaining_lease      113959 non-null  object 
 10  resale_price         113959 non-null  float64
dtypes: float64(2), int64(1), object(8)
memory usage: 9.6+ MB


In [5]:
df['Address'] = df['block'] + " " + df['street_name']
# Prints only every 20 entries, This reduces total dataset of 113959 to a much manageable number
# API only accepts certain format so having them fed into the API as 'Block number' 'Street Name ' works best
addresslist = list(df['Address'])[-10000:]

In [6]:
len(addresslist)

10000

In [7]:
df = df[-10000:]

In [8]:
df.head(10)

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,Address
103959,2021-08,ANG MO KIO,3 ROOM,556,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,57 years 06 months,305000.0,556 ANG MO KIO AVE 10
103960,2021-08,ANG MO KIO,3 ROOM,575,ANG MO KIO AVE 10,13 TO 15,67.0,New Generation,1980,57 years 06 months,318000.0,575 ANG MO KIO AVE 10
103961,2021-08,ANG MO KIO,3 ROOM,418,ANG MO KIO AVE 10,07 TO 09,74.0,New Generation,1979,57 years,353000.0,418 ANG MO KIO AVE 10
103962,2021-08,ANG MO KIO,3 ROOM,557,ANG MO KIO AVE 10,10 TO 12,68.0,New Generation,1980,57 years 05 months,328000.0,557 ANG MO KIO AVE 10
103963,2021-08,ANG MO KIO,3 ROOM,587,ANG MO KIO AVE 3,01 TO 03,67.0,New Generation,1979,56 years 11 months,300000.0,587 ANG MO KIO AVE 3
103964,2021-08,ANG MO KIO,3 ROOM,233,ANG MO KIO AVE 3,10 TO 12,67.0,New Generation,1977,55 years 02 months,315000.0,233 ANG MO KIO AVE 3
103965,2021-08,ANG MO KIO,3 ROOM,103,ANG MO KIO AVE 3,04 TO 06,74.0,New Generation,1978,55 years 07 months,326000.0,103 ANG MO KIO AVE 3
103966,2021-08,ANG MO KIO,3 ROOM,329,ANG MO KIO AVE 3,07 TO 09,73.0,New Generation,1978,55 years 05 months,360000.0,329 ANG MO KIO AVE 3
103967,2021-08,ANG MO KIO,3 ROOM,119,ANG MO KIO AVE 3,07 TO 09,67.0,New Generation,1978,56 years 03 months,333000.0,119 ANG MO KIO AVE 3
103968,2021-08,ANG MO KIO,3 ROOM,133,ANG MO KIO AVE 3,04 TO 06,73.0,New Generation,1979,56 years 06 months,328000.0,133 ANG MO KIO AVE 3


In [9]:
addresslist[:10]

['556 ANG MO KIO AVE 10',
 '575 ANG MO KIO AVE 10',
 '418 ANG MO KIO AVE 10',
 '557 ANG MO KIO AVE 10',
 '587 ANG MO KIO AVE 3',
 '233 ANG MO KIO AVE 3',
 '103 ANG MO KIO AVE 3',
 '329 ANG MO KIO AVE 3',
 '119 ANG MO KIO AVE 3',
 '133 ANG MO KIO AVE 3']

In [ ]:
addresslist = ['364A UPPER SERANGOON ROAD', '365B UPPER SERANGOON ROAD' , '365A UPPER SERANGOON ROAD', '119 SERANGOON NORTH AVE 1']

In [10]:
len(df)

10000

In [11]:
all_coordinates = []

for index, address in enumerate(addresslist):
    print('Progress {} percent \n'.format(np.round(index/len(addresslist)*100,2)))
    print('Searching for ', address)
    query = 'https://nominatim.openstreetmap.org/search?q=' + address + '&countrycodes=sg&limit=1&format=json'
    res = requests.get(query)
    result = eval(res.text)
    try:
        lat = result[0]['lat']
        long = result[0]['lon']
        print('Lat is ', lat)
        print('Long is ' , long)
        all_coordinates.append([lat, long])
        print('Information Found and appended \n')
        print('*'*25 , '\n')
    except:
        try:
            print('No results from openstreetmap, searching from onemap.sg')
            second_query = 'https://developers.onemap.sg/commonapi/search?searchVal='+ address +'&returnGeom=Y&getAddrDetails=Y&pageNum=1'
            res = requests.get(second_query)
            result = eval(res.text)
            lat = result['results'][0]['LATITUDE']
            lon = result['results'][0]['LONGITUDE']
            print('Lat is ', lat)
            print('Long is ' , long)
            all_coordinates.append([lat, long])
            print('Information Found and appended \n')
            print('*'*25 , '\n')
        except:
            print('Lat and long is unavailable')
            all_coordinates.append([0, 0])
            print('Information missing and appended null values \n')
            print('*'*25 , '\n')
            continue


Progress 0.0 percent 

Progress 0.01 percent 

Progress 0.02 percent 

Progress 0.03 percent 

Progress 0.04 percent 

Progress 0.05 percent 

Progress 0.06 percent 

Progress 0.07 percent 

Progress 0.08 percent 

Progress 0.09 percent 

Progress 0.1 percent 

Progress 0.11 percent 

Progress 0.12 percent 

Progress 0.13 percent 

Progress 0.14 percent 

Progress 0.15 percent 

Progress 0.16 percent 

Progress 0.17 percent 

Progress 0.18 percent 

Progress 0.19 percent 

Progress 0.2 percent 

Progress 0.21 percent 

Progress 0.22 percent 

Progress 0.23 percent 

Progress 0.24 percent 

Progress 0.25 percent 

Progress 0.26 percent 

Progress 0.27 percent 

Progress 0.28 percent 

Progress 0.29 percent 

Progress 0.3 percent 

Progress 0.31 percent 

Progress 0.32 percent 

Progress 0.33 percent 

Progress 0.34 percent 

Progress 0.35 percent 

Progress 0.36 percent 

Progress 0.37 percent 

Progress 0.38 percent 

Progress 0.39 percent 

Progress 0.4 percent 

Progress 0.41 percent

In [12]:
df_coordinates = pd.DataFrame(all_coordinates)#.reset_index(drop=True, inplace=True)#.astype(float)
print(df_coordinates)

               0                   1
0      1.3757477         103.8546396
1      1.3757477         103.8546396
2     1.36493535   103.8540607691629
3      1.3757477         103.8546396
4      1.3701783  103.85246795073925
...          ...                 ...
9995  1.42057595  103.83240578746648
9996  1.42016915  103.83595516211429
9997  1.41974085   103.8361781981992
9998   1.4143243          103.838877
9999  1.41900665  103.84058447202246

[10000 rows x 2 columns]


In [1]:
# you may not want to reset the index if you are doing it in its entirely or if you are doing the first part of the data
# however it is necessary to reset the index if you are doing it piecemeal, otherwise the coordinates retrieved will be lost
df = df.reset_index(drop=True)

NameError: name 'df' is not defined

In [14]:
df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,Address
0,2021-08,ANG MO KIO,3 ROOM,556,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,57 years 06 months,305000.0,556 ANG MO KIO AVE 10
1,2021-08,ANG MO KIO,3 ROOM,575,ANG MO KIO AVE 10,13 TO 15,67.0,New Generation,1980,57 years 06 months,318000.0,575 ANG MO KIO AVE 10
2,2021-08,ANG MO KIO,3 ROOM,418,ANG MO KIO AVE 10,07 TO 09,74.0,New Generation,1979,57 years,353000.0,418 ANG MO KIO AVE 10
3,2021-08,ANG MO KIO,3 ROOM,557,ANG MO KIO AVE 10,10 TO 12,68.0,New Generation,1980,57 years 05 months,328000.0,557 ANG MO KIO AVE 10
4,2021-08,ANG MO KIO,3 ROOM,587,ANG MO KIO AVE 3,01 TO 03,67.0,New Generation,1979,56 years 11 months,300000.0,587 ANG MO KIO AVE 3


In [15]:
df_coordinates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       10000 non-null  object
 1   1       10000 non-null  object
dtypes: object(2)
memory usage: 156.4+ KB


In [16]:
df_combined = df.join(df_coordinates)

In [17]:
print(df_combined.head())

     month        town flat_type block        street_name storey_range  \
0  2021-08  ANG MO KIO    3 ROOM   556  ANG MO KIO AVE 10     07 TO 09   
1  2021-08  ANG MO KIO    3 ROOM   575  ANG MO KIO AVE 10     13 TO 15   
2  2021-08  ANG MO KIO    3 ROOM   418  ANG MO KIO AVE 10     07 TO 09   
3  2021-08  ANG MO KIO    3 ROOM   557  ANG MO KIO AVE 10     10 TO 12   
4  2021-08  ANG MO KIO    3 ROOM   587   ANG MO KIO AVE 3     01 TO 03   

   floor_area_sqm      flat_model  lease_commence_date     remaining_lease  \
0            68.0  New Generation                 1980  57 years 06 months   
1            67.0  New Generation                 1980  57 years 06 months   
2            74.0  New Generation                 1979            57 years   
3            68.0  New Generation                 1980  57 years 05 months   
4            67.0  New Generation                 1979  56 years 11 months   

   resale_price                Address           0                   1  
0      305000

In [18]:
df_combined  = df_combined.rename(columns={0:'Latitude', 1:'Longitude'})

In [19]:
df_combined.to_csv('resalehdb_data_withCoordinates_last10K.csv',index=False)